In [ ]:
#import relevant libraries
import matplotlib.pyplot as plt
from statistics import mean
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import models,layers
import seaborn as sns 
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#hyperparameter tuning
HP_epochs = 30
HP_learning_rate = 0.000001
HP_batch_size = 1

HP_optimizer = keras.optimizers.Adam(learning_rate = HP_learning_rate)
HP_loss = 'sparse_categorical_crossentropy'

In [ ]:
#numeric representation of image
def get_arrays(folder):
    arr = []
    for i in os.listdir(folder):
        img = Image.open(os.path.join(folder,i))
        img_arr = np.array(img)
        rshp_arr = img_arr.reshape(-1,1)
        arr.append(rshp_arr)
    arr = np.squeeze(arr)
    return np.array(arr)

In [ ]:
#initializing input data
x = get_arrays("/content/drive/MyDrive/GAIP_Project/AudioClassification/Spectrogram")
print("Shape of input data is:",x.shape)
print(x)

In [ ]:
#initializing output data
y = np.array([0]*227+[1]*318)
print("Shape of output data is:",y.shape)
print(y)

In [ ]:
#scaling and feature reduction
se = StandardScaler()
x = se.fit_transform(x)
pca = PCA(0.90)
pca.fit(x)
pc_x = pca.transform(x)

print("Shape of input data is: ",pc_x.shape)
print(x)

In [ ]:
#splitting data into train and test
X_train,X_test,y_train,y_test = train_test_split(pc_x,y,train_size = 0.7,random_state = 2022)

In [ ]:
#constructing model
network=models.Sequential()
network.add(layers.Dense(400,activation='relu'))
network.add(layers.Dense(200,activation='relu'))
network.add(layers.Dense(2,activation='softmax'))

In [ ]:
#compiling model
network.compile(optimizer = HP_optimizer,loss = HP_loss,metrics = ['accuracy'])

In [ ]:
#model training and fit
history = network.fit(X_train,y_train.reshape(-1,1),epochs = HP_epochs,batch_size = HP_batch_size,validation_data = (X_test,y_test.reshape(-1,1)),verbose = False)

In [ ]:
#visualization
acc = history.history['accuracy'] 
val_acc = history.history['val_accuracy']

loss = history.history['loss'] 
val_loss = history.history['val_loss']

epochs_range = range(HP_epochs)

plt.figure(figsize=(8,6)) 
plt.plot(epochs_range, acc, label='Training Accuracy') 
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right') 
plt.title('Training and Validation Accuracy')
plt.show()

plt.figure(figsize=(8,6)) 
plt.plot(epochs_range, loss, label='Training Loss') 
plt.plot(epochs_range, val_loss, label='Validation Loss') 
plt.legend(loc='upper right') 
plt.title('Training and Validation Loss') 
plt.show()

In [ ]:
#training data prediction
preds = network.predict(X_train)
preds = np.argmax(preds,axis=1)


#test data prediction
test_preds = network.predict(X_test)
test_preds = np.argmax(test_preds,axis=1)

In [ ]:
#confusion matrix for training data
sns.heatmap(confusion_matrix(y_train,preds),annot=True)
acc = accuracy_score(y_train,preds)
acc

In [ ]:
#confusion matrix for testing data
sns.heatmap(confusion_matrix(y_test,test_preds),annot=True)
val_acc = accuracy_score(y_test,test_preds)
val_acc

In [ ]:
#network summary and metrics evaluation
report = classification_report( y_test, test_preds)
print(network.summary())
print(report)

In [ ]:
#prediction analysis
person={
    0: 'robot',
    1: 'human'
}
for i in range(len(test_preds)):
  print(f"Actual: {person[y_test[i]]}; Predicted: {person[test_preds[i]]}")

In [ ]:
#saving model
network.save("/content/drive/MyDrive/GAIP_Project/AudioClassification/audio_model.h5", history)
print("Model Saved")